In [ ]:
! pip install deeppavlov
! pip install transformers

In [ ]:
from deeppavlov import configs, train_model, build_model
from deeppavlov import evaluate_model
from deeppavlov.core.commands.utils import parse_config
from sklearn.model_selection import train_test_split
import json
import pandas as pd
from flask import Flask, jsonify, request
import joblib

#%%time
config = parse_config('/content/drive/MyDrive/ColabNotebooks/SkillBox/Configs/config_rubert_test_100_2.0.json')
model = build_model(config, download=False)

In [ ]:
def get_indexes(prob, top=3):
    sorted_indices = sorted(range(len(prob)), key=lambda k: prob[k], reverse=True)
    return sorted_indices[:top]

def get_top_intents(content, model, top=3):
    intents = list(model['classes_vocab'].keys())
    get_probs = model([content])[0][0]
    selected = [intents[i] for i in get_indexes(get_probs)]
    probabilities = [get_probs[i] for i in get_indexes(get_probs)]
    return pd.DataFrame({selected[0]: probabilities[0], selected[1]: probabilities[1], selected[2]: probabilities[2]})

In [ ]:
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)

    # Send back the result as dict
    return get_top_intents(data['content'], model)

if __name__ == '__main__':
    app.run(port=5000, debug=True)

In [1]:
! pip install fastapi[all] uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

#%%time
config = parse_config('/content/drive/MyDrive/ColabNotebooks/SkillBox/Configs/config_rubert_test_100_2.0.json')
model = build_model(config, download=False)

@app.route('/predict', methods=['POST'])
async def predict():
    data = request.get_json(force=True)

    # Send back the result as dict
    return get_top_intents(data['content'], model)